In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# from einops import rearrange, repeat
# import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
# from train import getSamples, getUpSamples, getSpacing


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
name = r'building'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
training_data = preProcess(data)
np.random.shuffle(training_data)

Preparing data ... (950/971)

In [6]:
sigmoid_ = nn.Sigmoid()
t = torch.tensor(np.linspace(-10, 10, 1000))
y = sigmoid_(t)
local_trans = 1 - y * (1 - y)
s = torch.cumprod(local_trans, 0)



In [5]:
x = np.random.rand(3,4)
print(x)
np.max(x, axis=0)

[[0.59267841 0.26816263 0.29743605 0.70478723]
 [0.69954568 0.57757708 0.38132738 0.00520854]
 [0.45320035 0.47800743 0.4549409  0.07791075]]


array([0.69954568, 0.57757708, 0.4549409 , 0.70478723])